To get sudokus
https://github.com/dimitri/sudoku/blob/master/sudoku.txt

# Import

In [1]:
import pandas as pd
import numpy as np
import sys

# Original

In [2]:
original = "003020600900305001001806400008102900700000008006708200002609500800203009005010300"
solution = "483921657967345821251876493548132976729564138136798245372689514814253769695417382"

# Functions

In [3]:
def string2df(s):
    return pd.DataFrame(np.array(list(s.replace('0', ' '))).reshape((9,9)))

In [4]:
def get_mutables(s):
    return list(np.where(np.array(list(s)) == '0')[0])

In [5]:
def _generate_init(s):
    return ''.join([x if x != '0' else str(np.random.randint(1,10)) for x in list(s)] )

In [6]:
def generate_inits(s, how_many=1000):
    return [_generate_init(s) for x in range(how_many)]

In [7]:
def _score_set_of_9(set_of_9):
    return 9 - np.unique(list(set_of_9)).shape[0]

In [8]:
def _get_combos(s):
    """ rows, columns and the suares """
    rows = [s[9*i:9*i+9] for i in range(9)]
    cols = [s[i:81:9] for i in range(9)]
    suares = [''.join([s[3*i:3*i+3] for i in range(x, x+9, 3)]) for x in range(21) if x%9 in [0,1,2]]
    return rows + cols + suares

In [9]:
def score_single(s):
    combos = _get_combos(s)
    return np.sum([_score_set_of_9(combo) for combo in combos])

In [10]:
def score_population(population):
    return [score_single(x) for x in population]

In [11]:
def _make_kids(p1, p2):
    random_cut = np.random.randint(1, len(p1))
    c1 = p1[:random_cut] + p2[random_cut:]
    c2 = p2[:random_cut] + p1[random_cut:]
    return c1, c2

In [12]:
def _get_parent_combis(parents):
    parent_combis = []
    for i in range(len(parents)):
        parent_combis.extend([(parents[i], x) for x in parents[i+1:]])

    return parent_combis

In [13]:
def make_many_kids(parents):
    parent_combis = _get_parent_combis(parents)
    
    kids = []
    for par_com in parent_combis:
        kids.extend(_make_kids(*par_com))

    return kids

In [14]:
def get_best(population, scores, top=45):
    return list(pd.Series(scores, index=population).sort_values().head(top).index)

In [15]:
def mutation(s, position):
    s = list(s)
    s[position] = str(np.random.randint(1,10))
    s = ''.join(s)
    return s

In [16]:
def mutate(original, to_mutate, number_of_mutations=1):
    mutables = get_mutables(original)
    positions = np.random.choice(mutables, size=number_of_mutations)
    for pos in positions:
        to_mutate = mutation(to_mutate, pos)
                             
    return to_mutate

In [17]:
def mutate_population(original, population, fraction=1, number_of_mutations=1):
    population = [mutate(original, x, number_of_mutations) if np.random.random()<fraction else x for x in population]
    return population

# Run

In [20]:
%%time
# Itiate some variables
final_solution = ''
scores = [100]  # just a high number to start with
population = generate_inits(original, 45)

# Let's make lot's of kids
for i in range(10**10):
    kids = make_many_kids(population)
    
    # Mutate kids in a smart fashion. Not too agressive at the end
    if np.min(scores) > 20:
        kids = mutate_population(original, kids, fraction=1, number_of_mutations=2)
    elif np.min(scores) > 6:
        kids = mutate_population(original, kids, fraction=1, number_of_mutations=1)
    else:
        kids = mutate_population(original, kids, fraction=.5, number_of_mutations=1)
        
    # score population
    population = np.unique(population + kids)
    scores = score_population(population)
    
    # Check if we found the solution
    if np.min(scores) == 0:
        final_solution = get_best(population, scores, 1)[0]
        sys.stdout.write(f'\r The solution has been found at iteration {i}. Ow yeah :D')
        break
    
    # Progessbar and new population
    sys.stdout.write(f'\r Iteration: {i} \t Best score: {np.min(scores)} \t Mean score: {np.mean(scores).round(2)}')
    population = get_best(population, scores)

 Iteration: 228 	 Best score: 10 	 Mean score: 12.74

KeyboardInterrupt: 

In [ ]:
print(final_solution==solution)
string2df(final_solution)

# End